In [ ]:
from chanlun.backtesting import backtest
from chanlun.backtesting.optimize import OptimizationSetting
from chanlun.strategy.my_strategy_hs300_tupo_debug import StrategyTupo
import pandas as pd
from chanlun.cl_utils import query_cl_chart_config
save_path = 'D:/xuangu_log/a_strategy_hs300_tupo_debug/'

In [ ]:
zhishu = [
    "SHSE.000016",  # 上证50
    "SHSE.000044",  # 上证中盘
    "SZSE.399330", # 深圳100
]

In [ ]:
# 量化配置
bt_config = {
    # 策略结果保存的文件
    # 'save_file': 'D:/xuangu_log/a_strategy_Dnbbc/a_strategy_Dnbbc.pkl',
    'save_file': save_path + save_path.split('/')[-2] + '.pkl',

    # 设置策略对象
    'strategy': StrategyTupo(),
    # 回测模式：signal 信号模式，固定金额开仓； trade 交易模式，按照实际金额开仓
    'mode': 'signal',
    # 市场配置，currency 数字货币  a 沪深  hk  港股  futures  期货
    'market': 'a',
    # 基准代码，用于获取回测的时间列表
    'base_code': 'SHSE.000300',
    # 回测的标的代码，2021年后符合条件股票
    'codes' : ['SHSE.000300'], 
    # 'codes' : ['SH.600150', 'SH.600203'], 
    # 回测的周期，这里设置里，在策略中才能取到对应周期的数据
    'frequencys': ['1m'],
    # 'frequencys': ['d','1m'],
    # 回测开始的时间
    'start_datetime': '2023-09-05 00:00:00',
    # 'start_datetime': '2023-01-01 00:00:00',
    # 回测的结束时间
    'end_datetime': '2024-12-31 15:00:00',
    # mode 为 trade 生效，初始账户资金
    'init_balance': 10000000,
    # mode 为 trade 生效，交易手续费率
    'fee_rate': 0.0006,
    # mode 为 trade 生效，最大持仓数量（分仓）
    'max_pos': 10,
    # 缠论计算的配置，详见缠论配置说明
    'cl_config': query_cl_chart_config('a', 'SH.000001'),
}

# 参数优化设置
setting = OptimizationSetting()
setting.add_cl_parameter('bi_type', ['bi_type_old', 'bi_type_new'])
setting.add_cl_parameter('bi_bzh', ['bi_bzh_yes', 'bi_bzh_no'])
setting.add_cl_parameter('zs_bi_type', ['zs_type_bz', 'zs_type_dn'])
setting.add_cl_parameter('zs_xd_type', ['zs_type_bz', 'zs_type_dn'])


BT = backtest.BackTest(bt_config)
BT.can_close_today = True  # 是否可以平今
BT.can_short = True  # 是否可以做空
BT.datas.load_data_to_cache = True # 是否将所有k线数据加载到内存中，如果内存吃紧，可以设置为 False，这样每次循环都会从数据库中读取
BT.datas.load_kline_nums = 100000 # 每次 klines 方法中返回的k线数据条数
BT.datas.cl_data_kline_max_nums = 300000 # 回测过程中，如果缠论对象的数据量超过这个值，则会清空重新计算，cl_data_kline_max_nums 不能小于 load_kline_nums，否则将会不断的删除并重新计算

In [ ]:
# # 跑参数优化
# results = BT.run_optimization(setting)
# print(results)

In [ ]:
# # 跑参数优化
# results = BT.run_optimization(setting)
# print(results)

In [ ]:
# 运行回测
BT.run(BT.frequencys[-1])
# 多进程回测，只支持信号回测模式
# BT.run_process(BT.frequencys[-1], 22)
# 保存回测结果到文件中
BT.save()
print('Done')
# 10m 57.59s

In [ ]:
# 输出回测结果
BT.info()
BT.result()

# 信号模式结果，总盈利 110W
# |   一类买点   |  4   |  7   | 36.36% |  89163.19 |  23073.57 |  66089.62 |  25.88   | 22290.8  | 3296.22  |  6.7625 |
# |   二类买点   |  2   |  8   | 20.0%  |  65978.99 |  50110.35 |  15868.64 |  75.95   | 32989.49 | 6263.79  |  5.2667 |
# |   三类买点   |  34  |  34  | 50.0%  | 872771.93 | 235193.11 | 637578.82 |  26.95   | 25669.76 | 6917.44  |  3.7109 |
# |  类三类买点  |  20  |  24  | 45.45% | 368807.73 | 155924.38 | 212883.35 |  42.28   | 18440.39 | 6496.85  |  2.8384 |
# | 下跌盘整背驰 |  2   |  5   | 28.57% | 211139.01 |  34745.79 | 176393.21 |  16.46   | 105569.5 | 6949.16  | 15.1917 |

# 交易模式 最大持仓 10，总盈利 65W
# 首个交易日：2017-01-03 00:00:00 最后交易日：2022-04-20 00:00:00 总交易日：1287
# 起始资金：1000000 结束资金：1,656,654.08 总手续费：16,188.58
# 基准收益率：1.47%  基准年化收益：0.27%
# 总收益率：65.67% 年化收益率：12.25%
# 最大回撤：-220,695.11 百分比最大回撤：-19.14% 最长回撤天数：789
# 日均收益率：0.04% 收益标准差：1.06% Sharpe Ratio: 0.54 收益回撤比：3.43 
# |   一类买点   |  4   |  6   | 40.0%  | 117552.34 |  28990.72 |  88561.61 |  24.66   |  29388.08 | 4831.79  |  6.0822 |
# |   二类买点   |  2   |  7   | 22.22% |  97119.24 |  59261.18 |  37858.06 |  61.02   |  48559.62 | 8465.88  |  5.7359 |
# |   三类买点   |  25  |  30  | 45.45% | 468874.21 | 221022.02 | 247852.19 |  47.14   |  18754.97 |  7367.4  |  2.5457 |
# |  类三类买点  |  15  |  22  | 40.54% | 323706.58 |  192148.1 | 131558.48 |  59.36   |  21580.44 |  8734.0  |  2.4709 |
# | 下跌盘整背驰 |  1   |  4   | 20.0%  | 193717.31 |  27356.51 | 166360.81 |  14.12   | 193717.31 | 6839.13  | 28.3249 |

In [ ]:
# 显示净盈利变化曲线
BT.backtest_charts()

In [ ]:
pos_df = BT.positions()
pos_df['open_datetime'] = pd.to_datetime(pos_df['open_datetime']).dt.tz_localize(None)
pos_df['close_datetime'] = pd.to_datetime(pos_df['close_datetime']).dt.tz_localize(None)
excel_file = save_path + save_path.split('/')[-2] + '.xlsx'
print(excel_file)
pos_df.to_excel(excel_file)
# pos_df.to_csv("positions_30m_direct.csv")

In [ ]:
# 显示历史持仓
#设置显示全部行，不省略
pd.set_option('display.max_rows',None)
#设置显示全部列，不省略
pd.set_option('display.max_columns',None)

pos_df = BT.positions()
pos_df = pos_df.sort_values(['profit_rate'])
pos_codes = pos_df['code'].unique()

print(len(pos_df), len(pos_df[pos_df['max_profit_rate'] >= 5]))
pos_df

In [ ]:
# 展示图表
code = pos_codes[0]
print(BT.positions(code))
# BT.show_charts(code, BT.frequencys[0], show_futu='macd')
BT.show_charts(code, BT.frequencys[0])

In [ ]:
# 展示图表
BT.show_charts(code, BT.frequencys[0])